In [57]:
from sae_lens import SAE, LanguageModelSAERunnerConfig, SAEConfig
from datasets import load_dataset, load_from_disk
from transformer_lens import HookedTransformer
import torch
import plotly.express as px

In [2]:
# token_dataset = load_dataset('taufeeque/othellogpt', split='train')
# sae = SAE.load_from_pretrained('othello_sae')

In [3]:
# batch_tokens = token_dataset[:32]["tokens"]
# print(batch_tokens)

In [4]:
# sae.eval()
# sae.cuda()
# with torch.no_grad():
#     # activation store can give us tokens.
#     batch_tokens = token_dataset[:32]["tokens"]
#     # convert to tensor
#     batch_tokens = torch.tensor(batch_tokens)[:, -1]
#     _, cache = model.run_with_cache(batch_tokens.cuda(), prepend_bos=False)
    
    
#     feature_acts = sae.encode(cache[sae.cfg.hook_name])
#     sae_out = sae.decode(feature_acts)
    
    
#     l0 = (feature_acts[:, 1:] > 0).float().sum(-1).detach()
#     print("average l0", l0.mean().item())
#     px.histogram(l0.flatten().cpu().numpy()).show()

In [5]:
# from transformer_lens import utils
# from functools import partial

# # next we want to do a reconstruction test.
# def reconstr_hook(activation, hook, sae_out):
#     return sae_out


# def zero_abl_hook(activation, hook):
#     return torch.zeros_like(activation)


# print("Orig", model(batch_tokens, return_type="loss").item())
# print(
#     "reconstr",
#     model.run_with_hooks(
#         batch_tokens,
#         fwd_hooks=[
#             (
#                 sae.cfg.hook_name,
#                 partial(reconstr_hook, sae_out=sae_out),
#             )
#         ],
#         return_type="loss",
#     ).item(),
# )
# print(
#     "Zero",
#     model.run_with_hooks(
#         batch_tokens,
#         return_type="loss",
#         fwd_hooks=[(sae.cfg.hook_name, zero_abl_hook)],
#     ).item(),
# )

In [17]:
COLAB = False
import webbrowser
import http.server
import platform

import socketserver
import threading
PORT = 8000
from IPython.display import IFrame, display
import subprocess


In [99]:
# create dummy tokenizer

import string

# Initialize the dictionary
vocab = {}

# Generate the keys (0 to 59) and the corresponding values (A1 to H8)
letters = string.ascii_uppercase[:8]  # Get letters 'A' to 'H'
index = 0

for letter in letters:
    for number in range(1, 9):  # Numbers 1 to 8
        vocab[index] = f"{letter}{number}"
        index += 1


In [120]:
from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData
import sae_vis
import importlib
importlib.reload(sae_vis)
importlib.reload(sae_vis.data_storing_fns)
from sae_vis import utils_fns
importlib.reload(utils_fns)
# importlib.reload(sae_vis.data_config_classes)
from sae_vis import data_fetching_fns
importlib.reload(data_fetching_fns)
import os
import torch as t
from transformer_lens import HookedTransformer, HookedTransformerConfig

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

sae = SAE.load_from_pretrained('trained_sae_my-own-othello-model_blocks.5.hook_resid_pre_5_32_5.0_512_102400000_0.001_5.0')
print(sae.cfg.d_sae)



device = t.device("cuda" if t.cuda.is_available() else "cpu")


n_layers = 6
d_model = 128

model_cfg = HookedTransformerConfig(
    n_layers=n_layers,
    d_model=d_model,
    d_head=64,
    n_heads=8,
    d_mlp=d_model * 4,
    d_vocab=61,
    n_ctx=59,
    act_fn="gelu",
    normalization_type="LNPre",
    device=device,
)


# model = HookedTransformer.from_pretrained('my-own-othello-model', fold_ln=True)
model = HookedTransformer(model_cfg).to(device)
model.load_and_process_state_dict(t.load(f"othello_gpt_6_128_lr0.0005_bs512_epochs2.pth"), fold_ln=False)

model.work_around_vocab = vocab

# batch_tokens = token_dataset[:32]["tokens"]
# convert to tensor
# batch_tokens = torch.tensor(batch_tokens)[:, -1]

test_feature_idx_gpt = range(0, 100)
hook_name = sae.cfg.hook_name


feature_vis_config_gpt = SaeVisConfig(
    hook_point=hook_name,
    features=test_feature_idx_gpt,
    batch_size=10_000, # just use the full sequence
    minibatch_size_tokens=512,
    verbose=True,
)
# token_dataset = load_dataset('taufeeque/othellogpt', split='train')
# print(type(token_dataset))
# print(token_dataset[:32]["tokens"])
# convert to tensor
# token_dataset = torch.tensor(token_dataset[:32]["tokens"])[:, :-1].cuda()
# print(token_dataset.shape)
# print(token_dataset[:32]["tokens"])
token_dataset = load_from_disk("data/validation_processed_othellogpt_dataset")['tokens']
# print(token_dataset['tokens'])
sae.eval()

sae_vis_data_gpt = SaeVisData.create(
    encoder=sae.cuda(),
    model=model, # type: ignore
    tokens=token_dataset,  # type: ignore
    cfg=feature_vis_config_gpt,
)

4096
Moving model to device:  cuda
Creating SaeVisData
Creating SaeVisData
Getting feature data
objects created
torch.Size([200000, 59])
torch.Size([10000, 59])
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
feature_batches created
























































































Forward passes to cache data for vis: 100%|██████████| 2/2 [02:01<00:00, 60.67s/it]






















































































































































































┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ Task                                           ┃ Time  ┃ Pct % ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ (1) Initialization                             │ 0.01s │ 0.0%  │
│ (2) Forward passes to gather model activations │ 2.83s │ 18.6% │
│ (3) Computing feature acts from model acts     │ 0.08s │ 0.5%  │
│ (4) Getting data for tables                    │ 0.01s │ 0.0%  │
│ (5) Getting data for histograms                │ 3.09s │ 20.3% │
│ (6) Getting data for sequences                 │ 8.88s │ 58.3% │
│ (7) Getting data for quantiles                 │ 0.34s │ 2.2%  │
└────────────────────────────────────────────────┴───────┴───────┘

Extracting vis data from cached data: 100%|██████████| 100/100 [00:16<00:00,  6.13it/s]


In [115]:
max = sae_vis_data_gpt.feature_data_dict[3_000].sequence_data
from pprint import pprint
pprint(max)

SequenceMultiGroupData(seq_group_data=[SequenceGroupData(title='TOP '
                                                               'ACTIVATIONS<br>MAX '
                                                               '= 5.390',
                                                         seq_data=[SequenceData(token_ids=[25,
                                                                                           53,
                                                                                           4,
                                                                                           24,
                                                                                           21,
                                                                                           54,
                                                                                           60,
                                                                                           9,
             

In [98]:
def display_vis_inline(filename: str, height: int = 850):
    '''
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    '''
    if not COLAB:
        if os.path.isfile(filename):
            print(f"Opening {filename}")
            # Print current folder
            print(f"Current folder: {os.getcwd()}")
            # Get the absolute path
            file_path = os.path.abspath(filename)
    
            # Check if running under WSL
            if 'microsoft-standard' in platform.uname().release:
                # Use the Windows default browser to open the file
                windows_path = subprocess.check_output(['wslpath', '-w', file_path]).decode().strip()
                subprocess.run(['explorer.exe', windows_path])
            else:
                # For other environments, use the default web browser
                webbrowser.open(f"file://{file_path}")
            
            # Display inline in Jupyter Notebook
            display(IFrame(src=f"file://{file_path}", width="100%", height=height))
        else:
            print(f"File not found: {filename}")

In [83]:
for feature in test_feature_idx_gpt[:1]:
    filename = f"{feature}_feature_vis_demo_gpt.html"
    sae_vis_data_gpt.save_feature_centric_vis(filename, feature)
    display_vis_inline(filename)

Saving feature-centric vis: 100%|██████████| 25/25 [00:03<00:00,  8.03it/s]


Opening 100_feature_vis_demo_gpt.html
Current folder: /mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens
